In [ ]:
import numpy as np
from itertools import product
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from mindreadingautobots.entropy_and_bayesian import boolean


### Brute-force for $n \leq 4$
We are looking for a case where 
$$
s(f) < s(g^*)
$$
where $g^*$ is the maximum likelihood predictor for the noisy data $(Z, f(X))$.

In [ ]:
# n = 5


n = 4
assert n <= 4
k = n
# pvals = [0.01, 0.25, 0.49]
pvals = [0.49]

# a "signature" of a boolean function is a length 2**n bitstring S where f(x) = S[bin(x)]
sig_arr = np.array(list(itertools.product([0, 1], repeat=2 ** n)))
X_arr = np.array(list(itertools.product([0, 1], repeat=n)))
p_x = 1 / (2 ** k) # uniform distribution over x
for i, signature in enumerate(sig_arr):
    # iterate over signatures
    dct, func = boolean.boolean_function_from_signature(signature)
    for p in pvals:        
        # noisy_lookup[row,col] is the JOINT probability Pr(f(z)=row| x=col)
        noisy_lookup = np.zeros((2, 2**n))
        true_lookup = np.zeros((2, 2**n))
        # simulate a noisy dataset essentially
        for i, x in enumerate(product([0,1], repeat=k)):
            func_value = func(x)
            # true lookup is an array with 2 rows; there is a p_x at [row, column] if 
            # f[column] = row]. so, true_lookup[i, j] = pr(f(x) = i| x=j)
            true_lookup[func(x), i] = 1
            # iterate over all of the z values that contribute to 
            for e in product([0, 1], repeat=k):
                z = np.array(x) ^ np.array(e)
                p_x_given_z = p ** sum(e) * (1-p)**(k - sum(e))
                # increment noisy_lookup at the binary index of z
                # noisy_lookup[i, j] = pr(f(z) = i,  x=j) 
                noisy_lookup[func_value, int(''.join(map(str, z)), 2)] += p_x_given_z 
        
        # the function is balanced if the sums of the two rows of true_lookup are equal
        # imbal = abs(true_lookup[0,:].sum() - true_lookup[1,:].sum())  / 2 ** n
        # round up to get argmax 
        noisy_mle = np.round(noisy_lookup)  
        out = np.multiply(noisy_mle, true_lookup) / 2 ** n # "inner product" of the functions
        diff = out.sum()
        fnstar_dct = {}
        for i, x in enumerate(X_arr):
            fnstar_dct[tuple(x)] = np.argmax(noisy_lookup[:, i])
        def fnstar(x):
            return fnstar_dct[tuple(x)]
        
        # compute sensitivity at i of f and fnstar to check for unformity of bound?
        # for kk in range(n):
        #     sensitivity_i_f = boolean.average_s_i(func, kk, X_arr)
        #     sensitivity_i_fnstar = boolean.average_s_i(fnstar, kk, X_arr)
        #     if sensitivity_i_f < sensitivity_i_fnstar:
        #         sensitivity_i_f = boolean.average_s_i(func, kk, X_arr, verbose=True)
        #         sensitivity_i_fnstar = boolean.average_s_i(fnstar, kk, X_arr, verbose=True)
        #         print(f"ERROR: fn* has higher sensitivity than f at bit {kk}")
        #         print(true_lookup)
        #         print(noisy_mle)
        #         for k, v in dct.items():
        #             print(k, v)

        #         raise ValueError

        sensitivity_f = boolean.average_sensitivity(func, X_arr)
        sensitivity_fnstar = boolean.average_sensitivity(fnstar, X_arr)
        sensitivity_diff = sensitivity_f - sensitivity_fnstar
        # accuracies on dataset
        p_zy = boolean.generate_noisy_distr(k, p, func)
        noisy_f_acc = boolean.compute_acc_noisytest(p_zy, func, n) # accuracy of f on noisy data
        # noiseless_fnstar_acc = compute_acc_test(fnstar, func, n) # accuracy of fN* on noiseless data
        noisy_fnstar_acc = boolean.compute_acc_noisytest(p_zy, fnstar, n) # accuracy of fN* MLE on noisy data
        if sensitivity_f < sensitivity_fnstar:
            print("ERROR: fn* has lower sensitivity than f")
            raise ValueError


i: 0, x: [0 0 0 0], s_i: 1
i: 0, x: [0 0 0 1], s_i: 0
i: 0, x: [0 0 1 0], s_i: 1
i: 0, x: [0 0 1 1], s_i: 1
i: 0, x: [0 1 0 0], s_i: 1
i: 0, x: [0 1 0 1], s_i: 1
i: 0, x: [0 1 1 0], s_i: 1
i: 0, x: [0 1 1 1], s_i: 0
i: 0, x: [1 0 0 0], s_i: 1
i: 0, x: [1 0 0 1], s_i: 0
i: 0, x: [1 0 1 0], s_i: 1
i: 0, x: [1 0 1 1], s_i: 1
i: 0, x: [1 1 0 0], s_i: 1
i: 0, x: [1 1 0 1], s_i: 1
i: 0, x: [1 1 1 0], s_i: 1
i: 0, x: [1 1 1 1], s_i: 0
i: 0, x: [0 0 0 0], s_i: 1
i: 0, x: [0 0 0 1], s_i: 1
i: 0, x: [0 0 1 0], s_i: 1
i: 0, x: [0 0 1 1], s_i: 1
i: 0, x: [0 1 0 0], s_i: 1
i: 0, x: [0 1 0 1], s_i: 1
i: 0, x: [0 1 1 0], s_i: 1
i: 0, x: [0 1 1 1], s_i: 1
i: 0, x: [1 0 0 0], s_i: 1
i: 0, x: [1 0 0 1], s_i: 1
i: 0, x: [1 0 1 0], s_i: 1
i: 0, x: [1 0 1 1], s_i: 1
i: 0, x: [1 1 0 0], s_i: 1
i: 0, x: [1 1 0 1], s_i: 1
i: 0, x: [1 1 1 0], s_i: 1
i: 0, x: [1 1 1 1], s_i: 1
ERROR: fn* has higher sensitivity than f at bit 0
[[1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 

ValueError: 

#### Searching for counterexamples

Instead of defining boolean functions as $f:\{0,1\}^n \rightarrow \{0,1\}$, we define $f$ such that 
$$
f(x) = g(\text{wt}(x))
$$
what this does is, there are only $2^{n+1}$ possible functions $g$, instead of $2^{2^n}$ possible functions $f$. For example, if $n=3$, then a possible $g$ is 
\begin{equation}
     \text{wt}(x) \rightarrow \begin{cases}
        0 \rightarrow 0 \\
        1 \rightarrow 1 \\
        2 \rightarrow 0 \\
        3 \rightarrow 1
    \end{cases}
\end{equation}